# BitMEX Wallets in/outflow
Date: 5/20/20  
Author: Evan Azevedo  
Organization: Amberdata  

## Description
Exploring the question: observing the in/outflows from BitMEX wallets, is there some impact on price? 

## Research Question
Does the inflow of money into BitMEX wallets have any correlation with the price?

### Key information
* All wallet addresses start with BMEX (I think those are motor bikes?)
* BitMEX have a single daily withdraw. So these wallets can take a lot of deposits, but limited withdraw - similar to a High Yield Savings acct.

In [55]:
# import the packages
import os
import requests
import json
import datetime
import matplotlib.pyplot as plt
from dotenv import load_dotenv

In [86]:
def get_key():
    "Get the API key from an .env file"
    if ".env" not in os.listdir("../"):
        print("Configuring API Key...")
        key = input("Amberdata API Key: ")
        with open(".env", "w") as f:
            f.write(f"AMBERDATA_API_KEY={key}\n")

    load_dotenv(verbose=True)

    return {
        "AMBERDATA_API_KEY": os.getenv("AMBERDATA_API_KEY"),
        "AWS_ACCESS_KEY": os.getenv("AWS_ACCESS_KEY"),
        "AWS_SECRET_KEY": os.getenv("AWS_SECRET_KEY")
    }

api_key = get_key()["AMBERDATA_API_KEY"]

In [77]:
def get_response(url, queryString=None):
    "Get the REST response from the specified URL"
    headers = {'x-api-key': api_key}
    if queryString:
        response = requests.request("GET", url, headers=headers, params=queryString)
    else:
        response = requests.request("GET", url, headers=headers)
    response = json.loads(response.text)
    if response["title"] == "OK":
        return response["payload"]
    else:
        print("Unsuccessful request.")

In [80]:
# get exchanges
url = "https://web3api.io/api/v2/market/exchanges"

payload = get_response(url=url)

In [39]:
print(payload.keys())

dict_keys(['binance', 'bitfinex', 'bithumb', 'bitstamp', 'gdax', 'gemini', 'huobi', 'kraken', 'okex', 'poloniex', 'zb', 'bitmex'])


In [42]:
bitmex = payload["bitmex"]
print(f"Bitmex pairs: {bitmex.keys()}")

Bitmex pairs: dict_keys(['adam20_bitmex', 'bchm20_bitmex', 'eosm20_bitmex', 'ethm20_bitmex', 'ethusd_bitmex', 'ethusdm20_bitmex', 'ltcm20_bitmex', 'trxm20_bitmex', 'xbtm20_bitmex', 'xbtu20_bitmex', 'xbtusd_bitmex', 'xrpm20_bitmex', 'xrpusd_bitmex', 'adau19_bitmex', 'bchu19_bitmex', 'eosu19_bitmex', 'ethu19_bitmex', 'ltcu19_bitmex', 'trxu19_bitmex', 'xbt7d_d95_bitmex', 'xbt7d_u105_bitmex', 'xbtu19_bitmex', 'xbtz19_bitmex', 'xrpu19_bitmex'])


In [81]:
url = "https://web3api.io/api/v2/market/ohlcv/information"

querystring = {"exchange":"bitmex"}

payload = get_response(url, querystring)
    
bitmex = payload["bitmex"]
print(f"Bitmex pairs: {bitmex.keys()}")

Bitmex pairs: dict_keys(['adam20_bitmex', 'bchm20_bitmex', 'eosm20_bitmex', 'ethm20_bitmex', 'ethusd_bitmex', 'ethusdm20_bitmex', 'ltcm20_bitmex', 'trxm20_bitmex', 'xbtm20_bitmex', 'xbtu20_bitmex', 'xbtusd_bitmex', 'xrpm20_bitmex', 'xrpusd_bitmex'])


In [47]:
pairs = []

for key in bitmex.keys():
    pairs.append(key.split("_")[0])

In [49]:
pair = 'ethusd_bitmex'

In [72]:
startDate = int(datetime.datetime(2020,5,1,0,0).timestamp())
endDate = int(datetime.datetime.now().timestamp())
print(startDate, endDate)

1588316400 1590079247


In [84]:
url = "https://web3api.io/api/v2/market/ohlcv/" + pair + "/historical"

querystring = {"exchange":"bitmex","startDate":startDate,"endDate":endDate,"timeInterval":"hours"}

payload = get_response(url, querystring)

In [85]:
payload

{'metadata': {'columns': ['timestamp',
   'open',
   'high',
   'low',
   'close',
   'volume'],
  'startDate': 1588316400000,
  'endDate': 1590079247000},
 'data': {'bitmex': [[1588316400000, 213.35, 214.1, 210.4, 212.9, 6064361],
   [1588320000000, 212.9, 213.95, 212.05, 213.75, 4144125],
   [1588323600000, 213.75, 215.3, 213.3, 215.15, 6822155],
   [1588327200000, 215.15, 215.7, 213.8, 215.6, 3873873],
   [1588330800000, 215.6, 217.8, 213.55, 215.65, 9718089],
   [1588334400000, 215.65, 215.95, 211.55, 213.3, 8412699],
   [1588338000000, 213.3, 214.15, 210.55, 211.35, 4604398],
   [1588341600000, 211.35, 212.8, 210.15, 212.5, 5481237],
   [1588345200000, 212.5, 213.2, 210.3, 210.3, 3730880],
   [1588348800000, 210.3, 211.65, 208.85, 210.4, 6503190],
   [1588352400000, 210.4, 211.4, 209.8, 211.3, 1964147],
   [1588356000000, 211.3, 212.1, 209.9, 210.1, 3584549],
   [1588359600000, 210.1, 210.4, 208.9, 209.45, 3269427],
   [1588363200000, 209.45, 210.35, 208.4, 210.15, 4620972],
   [1